In [1]:
import os
import clip
import torch
from PIL import Image
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR100
from torch.utils.data import Subset
from tqdm import tqdm
from torch.utils.data.dataset import Dataset
from torchvision import transforms
import random
from sklearn.model_selection import train_test_split, GridSearchCV

print(f'{torch.__version__}')


# 深度學習函式庫
import tensorflow
import tensorflow_addons as tfa
import tensorflow.keras as keras
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout, Activation, BatchNormalization
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
print(f'{tensorflow.__version__}')
# tensorflow.config.experimental_run_functions_eagerly = False

1.7.1


D:\anaconda3\envs\pytorch1.7.1_python3.8\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.6.0 and strictly below 2.9.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.4.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


2.4.0


In [104]:
features = np.load('Alldata_features_ViTL14.npy')
labels = np.load('Alldata_labels_ViTL14.npy')
train_features, val_features, train_labels, val_labels = train_test_split(features, labels,
                                                                           test_size=0.05,
                                                                           random_state=1,
                                                                           stratify=labels)
# train_features = np.load('train_features_ViTL14_DA5.npy')
# train_labels = np.load('train_labels_ViTL14_DA5.npy')
# val_features = np.load('val_features_ViTL14_DA5.npy')
# val_labels = np.load('val_labels_ViTL14_DA5.npy')

train_labels_OH = keras.utils.to_categorical(train_labels, num_classes=14)
val_labels_OH = keras.utils.to_categorical(val_labels, num_classes=14)

print(f'train_features.shape: {train_features.shape}')
print(f'train_labels.shape: {train_labels_OH.shape}')
print(f'val_features.shape: {val_features.shape}')
print(f'val_labels.shape: {val_labels_OH.shape}')

train_features.shape: (76256, 768)
train_labels.shape: (76256, 14)
val_features.shape: (4014, 768)
val_labels.shape: (4014, 14)


In [2]:
# test_data = np.load('test_features_ViTL14.npy')
# print(test_data.shape)

(20000, 768)


In [106]:
inputs = Input(shape=(768,)) # CLIP output 的Embedding維度

x = Dense(1024)(inputs) #1
x = BatchNormalization()(x)
x = Activation('relu')(x)

# x = Dense(4096)(inputs) #1
# x = BatchNormalization()(x)
# x = Activation('relu')(x)
x = Dropout(0.3)(x)

# 添加一个分类器
predictions = Dense(14, activation='softmax')(x)

# 构建我们需要训练的完整模型
model = Model(inputs=inputs, outputs=predictions)
print(model.summary())

# optimizer = Adam(lr=5e-5,decay=0.0001,beta_1=0.9,beta_2=0.999) #100e-6
optimizer = keras.optimizers.Nadam(lr=5e-5)
# optimizer = SGD(lr=1e-5, decay=1e-4, momentum=0.9, nesterov=True)
model.compile(loss = 'categorical_crossentropy',
#               loss = tfa.losses.SigmoidFocalCrossEntropy(),
              optimizer = optimizer,
              metrics = ['accuracy'])

epochs = 2000
aug_ratio = 1
batch_size = 2048
steps_per_epoch = int(aug_ratio * train_features.shape[0] / batch_size)
save_dir = os.path.join(os.getcwd(), 'saved_models') # 儲存模型的路徑
model_name = 'Model_CLIP.h5' # 模型名稱

if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)

checkpoint = ModelCheckpoint(model_path, monitor = 'val_loss', save_best_only = True, verbose = 1)
earlystop = EarlyStopping(monitor = 'val_loss', patience = 10, verbose = 1) # patience=10為連續10次模型loss沒再下降就停止

model_history = model.fit(train_features, train_labels_OH, 
                          batch_size = batch_size,
                          epochs = epochs,
                          verbose = True,
                          callbacks = [checkpoint, earlystop],
                          validation_data = (val_features, val_labels_OH),
                          steps_per_epoch=steps_per_epoch
)

Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        [(None, 768)]             0         
_________________________________________________________________
dense_28 (Dense)             (None, 1024)              787456    
_________________________________________________________________
batch_normalization_16 (Batc (None, 1024)              4096      
_________________________________________________________________
activation_16 (Activation)   (None, 1024)              0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_29 (Dense)             (None, 14)                14350     
Total params: 805,902
Trainable params: 803,854
Non-trainable params: 2,048
________________________________________________

37/37 [==============================] - 0s 12ms/step - loss: 0.0927 - accuracy: 0.9706 - val_loss: 0.0962 - val_accuracy: 0.9718

Epoch 00026: val_loss improved from 0.09771 to 0.09623, saving model to D:\SideProject\Techniques\AIdea農作物辨識\saved_models\Model_CLIP.h5
Epoch 27/2000
37/37 [==============================] - 0s 12ms/step - loss: 0.0914 - accuracy: 0.9713 - val_loss: 0.0945 - val_accuracy: 0.9721

Epoch 00027: val_loss improved from 0.09623 to 0.09450, saving model to D:\SideProject\Techniques\AIdea農作物辨識\saved_models\Model_CLIP.h5
Epoch 28/2000
37/37 [==============================] - 0s 12ms/step - loss: 0.0880 - accuracy: 0.9729 - val_loss: 0.0931 - val_accuracy: 0.9723

Epoch 00028: val_loss improved from 0.09450 to 0.09312, saving model to D:\SideProject\Techniques\AIdea農作物辨識\saved_models\Model_CLIP.h5
Epoch 29/2000
37/37 [==============================] - 0s 12ms/step - loss: 0.0850 - accuracy: 0.9736 - val_loss: 0.0921 - val_accuracy: 0.9721

Epoch 00029: val_loss impr


Epoch 00056: val_loss improved from 0.07590 to 0.07531, saving model to D:\SideProject\Techniques\AIdea農作物辨識\saved_models\Model_CLIP.h5
Epoch 57/2000
37/37 [==============================] - 0s 12ms/step - loss: 0.0483 - accuracy: 0.9862 - val_loss: 0.0750 - val_accuracy: 0.9761

Epoch 00057: val_loss improved from 0.07531 to 0.07505, saving model to D:\SideProject\Techniques\AIdea農作物辨識\saved_models\Model_CLIP.h5
Epoch 58/2000
37/37 [==============================] - 0s 12ms/step - loss: 0.0486 - accuracy: 0.9861 - val_loss: 0.0748 - val_accuracy: 0.9763

Epoch 00058: val_loss improved from 0.07505 to 0.07478, saving model to D:\SideProject\Techniques\AIdea農作物辨識\saved_models\Model_CLIP.h5
Epoch 59/2000
37/37 [==============================] - 1s 14ms/step - loss: 0.0472 - accuracy: 0.9864 - val_loss: 0.0741 - val_accuracy: 0.9768

Epoch 00059: val_loss improved from 0.07478 to 0.07409, saving model to D:\SideProject\Techniques\AIdea農作物辨識\saved_models\Model_CLIP.h5
Epoch 60/2000
37/37 

37/37 [==============================] - 0s 12ms/step - loss: 0.0263 - accuracy: 0.9937 - val_loss: 0.0685 - val_accuracy: 0.9781

Epoch 00091: val_loss improved from 0.06880 to 0.06851, saving model to D:\SideProject\Techniques\AIdea農作物辨識\saved_models\Model_CLIP.h5
Epoch 92/2000
37/37 [==============================] - 0s 12ms/step - loss: 0.0261 - accuracy: 0.9937 - val_loss: 0.0687 - val_accuracy: 0.9783

Epoch 00092: val_loss did not improve from 0.06851
Epoch 93/2000
37/37 [==============================] - 0s 13ms/step - loss: 0.0247 - accuracy: 0.9946 - val_loss: 0.0679 - val_accuracy: 0.9783

Epoch 00093: val_loss improved from 0.06851 to 0.06790, saving model to D:\SideProject\Techniques\AIdea農作物辨識\saved_models\Model_CLIP.h5
Epoch 94/2000
37/37 [==============================] - 0s 12ms/step - loss: 0.0250 - accuracy: 0.9944 - val_loss: 0.0685 - val_accuracy: 0.9781

Epoch 00094: val_loss did not improve from 0.06790
Epoch 95/2000
37/37 [==============================] - 0s 12

In [103]:
final_model = load_model(os.path.join(save_dir, model_name))
final_model.summary()
eval_predictions = final_model.evaluate(val_features, val_labels_OH)


Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 768)]             0         
_________________________________________________________________
dense_23 (Dense)             (None, 4096)              3149824   
_________________________________________________________________
batch_normalization_13 (Batc (None, 4096)              16384     
_________________________________________________________________
activation_13 (Activation)   (None, 4096)              0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 4096)              0         
_________________________________________________________________
dense_24 (Dense)             (None, 14)                57358     
Total params: 3,223,566
Trainable params: 3,215,374
Non-trainable params: 8,192
____________________________________________

## Test

In [57]:
model = load_model('saved_models/All_1024_512_NoDA_BN_DO_9804.h5')
model.summary()

Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        [(None, 768)]             0         
_________________________________________________________________
dense_26 (Dense)             (None, 1024)              787456    
_________________________________________________________________
batch_normalization_4 (Batch (None, 1024)              4096      
_________________________________________________________________
activation_14 (Activation)   (None, 1024)              0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_27 (Dense)             (None, 14)                14350     
Total params: 805,902
Trainable params: 803,854
Non-trainable params: 2,048
________________________________________________

In [92]:
predictions = model.predict(test_data)
predictions.shape

(20000, 14)

In [93]:
file_classes = {0:'banana', 1:'bareland', 2:'carrot', 3:'corn', 4:'dragonfruit',
                5:'garlic', 6:'guava', 7:'peanut', 8:'pineapple', 9:'pumpkin', 
                10:'rice',11:'soybean', 12:'sugarcane', 13:'tomato'}

In [94]:
pre_classes = np.argmax(predictions, axis=1)

In [95]:
pre_classes

array([ 3,  8,  1, ...,  1, 11,  5], dtype=int64)

In [96]:
submit_csv = pd.read_csv('Test/submission_example.csv')
submit_csv = submit_csv.iloc[:, 0:1]
submit_csv['label'] = pre_classes
submit_csv

,image_filename,label
0,0005479429a4.jpg,3
1,0006849c44f2.jpg,8
2,0006fcc93fc9.jpg,1
3,00088803914c.jpg,5
4,000b572940a1.jpg,7
...,...,...
19995,fff17dcac3ec.jpg,0
19996,fff5a3691463.jpg,6
19997,fff713c48e87.jpg,1
19998,fff8ecc6bd97.jpg,11


In [97]:
submit_csv['label'] = submit_csv['label'].map(file_classes)
submit_csv

,image_filename,label
0,0005479429a4.jpg,corn
1,0006849c44f2.jpg,pineapple
2,0006fcc93fc9.jpg,bareland
3,00088803914c.jpg,garlic
4,000b572940a1.jpg,peanut
...,...,...
19995,fff17dcac3ec.jpg,banana
19996,fff5a3691463.jpg,guava
19997,fff713c48e87.jpg,bareland
19998,fff8ecc6bd97.jpg,soybean


In [98]:
submit_csv.to_csv('submit.csv', index=None)